In [1]:
import pandas as pd

DATA = '/kaggle/input/lumbar-coordinate-pretraining-dataset/coords_pretrain.csv'
coords_df = pd.read_csv(filepath_or_buffer=DATA)
coords_df.head()

,filename,source,x,y,level,relative_x,relative_y
0,1_t2.jpg,spider,139,175,L5/S1,0.542969,0.683594
1,1_t2.jpg,spider,133,157,L4/L5,0.519531,0.613281
2,1_t2.jpg,spider,132,131,L3/L4,0.515625,0.511719
3,1_t2.jpg,spider,131,102,L2/L3,0.511719,0.398438
4,1_t2.jpg,spider,134,84,L1/L2,0.523438,0.328125


Let'd so some EDA before we proceed. Do our different sources have comparable photos in them?

In [2]:
import warnings
from plotly import express

warnings.filterwarnings(action='ignore', category=FutureWarning)
express.scatter(data_frame=coords_df.sample(frac=0.25, random_state=2024), x='relative_x', y='relative_y', color='source', hover_name='filename', height=800)

It doesn't look like the different sources contain very similar pictures; the TSEG pictures cluster tightly in the middle, while the LSD and SPIDER pictures are more spread out but in different directions.

In [3]:
express.scatter(data_frame=coords_df.sample(frac=0.25, random_state=2024), x='x', y='y', color='source', hover_name='filename', height=800)

We get essentially the same plot if whether we use x/y or their relative equivalents; it looks like they contain the same information.

In [4]:
express.scatter(data_frame=coords_df.sample(frac=0.25, random_state=2024), x='x', y='y', color='level', hover_name='filename', height=800)

If we plot the levels we see that the X/Y values predict the level, but not perfectly. This is probably not surprising.